https://www.kaggle.com/kostyaatarik/close-ends-chunks-optimization-aka-2-opt

In [24]:
import numpy as np
import pandas as pd
import numba
from sympy import isprime, primerange
from math import sqrt
from sklearn.neighbors import KDTree
from tqdm import tqdm_notebook as tqdm
from itertools import combinations

In [2]:
cities = pd.read_csv('input/cities.csv', index_col=['CityId'])
XY = np.stack((cities.X.astype(np.float32), cities.Y.astype(np.float32)), axis=1)
is_not_prime = np.array([0 if isprime(i) else 1 for i in cities.index], dtype=np.int32)

In [3]:
@numba.jit('f8(i8, i8, i8)', nopython=True, parallel=False)
def cities_distance(offset, id_from, id_to):
    xy_from, xy_to = XY[id_from], XY[id_to]
    dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
    distance = sqrt(dx * dx + dy * dy)
    if offset % 10 == 9 and is_not_prime[id_from]:
        return 1.1 * distance
    return distance


@numba.jit('f8(i4, i8[:])', nopython=True, parallel=False)
def score_chunk(offset, chunk):
    pure_distance, penalty = 0.0, 0.0
    penalty_modulo = 9 - offset % 10
    for path_index in numba.prange(chunk.shape[0] - 1):
        id_from, id_to = chunk[path_index], chunk[path_index+1]
        xy_from, xy_to = XY[id_from], XY[id_to]
        dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
        distance = sqrt(dx * dx + dy * dy)
        pure_distance += distance
        if path_index % 10 == penalty_modulo and is_not_prime[id_from]:
            penalty += distance
    return pure_distance + 0.1 * penalty


@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def score_path(path):
    return score_chunk(0, path)

In [4]:
kdt = KDTree(XY)

In [5]:
pairs = set()
for city_id in tqdm(cities.index):
    dists, neibs = kdt.query([XY[city_id]], 31)
    for neib_id in neibs[0][1:]:
        if city_id and neib_id:  # skip pairs that include starting city 
            pairs.add(tuple(sorted((city_id, neib_id))))
    neibs = kdt.query_radius([XY[city_id]], 31, count_only=False, return_distance=False)
    for neib_id in neibs[0]:
        if city_id and neib_id and city_id != neib_id:
            pairs.add(tuple(sorted((city_id, neib_id))))

print(f'{len(pairs)} cities pairs are selected.')
# sort pairs by distance
pairs = np.array(list(pairs))
distances = np.sum((XY[pairs.T[0]] - XY[pairs.T[1]])**2, axis=1)
order = distances.argsort()
pairs = pairs[order]

100%|██████████| 197769/197769 [01:04<00:00, 3088.24it/s]


6275850 cities pairs are selected.


In [6]:
path = np.array(pd.read_csv('cuda_submission.csv').Path)
initial_score = score_path(path)
initial_score

1516713.767614174

In [7]:
path_index = np.argsort(path[:-1])

total_score = initial_score
print(f'Total score is {total_score:.2f}.')
for _ in range(3):
    for step, (id1, id2) in enumerate(tqdm(pairs), 1):
        if step % 10**6 == 0:
            new_total_score = score_path(path)
            print(f'Score: {new_total_score:.2f}; improvement over last 10^6 steps: {total_score - new_total_score:.2f}; total improvement: {initial_score - new_total_score:.2f}.')
            total_score = new_total_score
        i, j = path_index[id1], path_index[id2]
        i, j = min(i, j), max(i, j)
        chunk, reversed_chunk = path[i-1:j+2], np.concatenate([path[i-1:i], path[j:i-1:-1], path[j+1:j+2]])
        chunk_score, reversed_chunk_score = score_chunk(i-1, chunk), score_chunk(i-1, reversed_chunk)
        if j - i > 2:
            chunk_abc = np.concatenate([path[i-1:i+1], path[j:i:-1], path[j+1:j+2]])
            chunk_acb = np.concatenate([path[i-1:i], path[j:j+1], path[i:j], path[j+1:j+2]])
            chunk_abcb = np.concatenate([path[i-1:i+1], path[j:j+1], path[i+1:j], path[j+1:j+2]])
            abc_score, acb_score, abcb_score = map(lambda chunk: score_chunk(i-1, chunk), [chunk_abc, chunk_acb, chunk_abcb])
            for chunk, score, name in zip((chunk_abc, chunk_acb, chunk_abcb), (abc_score, acb_score, abcb_score), ('abc', 'acb', 'abcb')):
                if score < chunk_score:
                    path[i-1:j+2] = chunk
                    path_index = np.argsort(path[:-1])  # update path index
                    chunk_score = score
        if reversed_chunk_score < chunk_score:
            path[i-1:j+2] = reversed_chunk
            path_index = np.argsort(path[:-1])  # update path index

  0%|          | 2244/6275850 [00:00<04:54, 21273.75it/s]

Total score is 1516713.77.


 16%|█▌        | 1002703/6275850 [00:53<04:39, 18891.72it/s]

Score: 1516531.60; improvement over last 10^6 steps: 182.17; total improvement: 182.17.


 32%|███▏      | 2001991/6275850 [02:07<04:31, 15739.76it/s]

...

 80%|███████▉  | 5001592/6275850 [07:05<01:48, 11747.77it/s]

Score: 1516514.55; improvement over last 10^6 steps: 0.00; total improvement: 199.22.


 96%|█████████▌| 6000929/6275850 [08:58<00:24, 11146.61it/s]

Score: 1516514.55; improvement over last 10^6 steps: 0.00; total improvement: 199.22.


100%|██████████| 6275850/6275850 [09:49<00:00, 10651.27it/s]


In [8]:
print(f'Total improvement is {initial_score - total_score:.2f}.')

Total improvement is 199.22.


In [9]:
pd.DataFrame({'Path': path}).to_csv('submission_close1.csv', index=False)

In [26]:
# https://www.kaggle.com/kostyaatarik/not-a-3-and-3-halves-opt
import numpy as np
import pandas as pd
import numba
from sympy import isprime, primerange
from math import sqrt
from sklearn.neighbors import KDTree
from tqdm import tqdm_notebook as tqdm
from itertools import combinations, permutations
from functools import lru_cache

In [27]:
cities = pd.read_csv('input/cities.csv', index_col=['CityId'])
XY = np.stack((cities.X.astype(np.float32), cities.Y.astype(np.float32)), axis=1)
is_not_prime = np.array([0 if isprime(i) else 1 for i in cities.index], dtype=np.int32)

In [28]:
@numba.jit('f8(i8, i8, i8)', nopython=True, parallel=False)
def cities_distance(offset, id_from, id_to):
    xy_from, xy_to = XY[id_from], XY[id_to]
    dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
    distance = sqrt(dx * dx + dy * dy)
    if offset % 10 == 9 and is_not_prime[id_from]:
        return 1.1 * distance
    return distance


@numba.jit('f8(i4, i8[:])', nopython=True, parallel=False)
def score_chunk(offset, chunk):
    pure_distance, penalty = 0.0, 0.0
    penalty_modulo = 9 - offset % 10
    for path_index in numba.prange(chunk.shape[0] - 1):
        id_from, id_to = chunk[path_index], chunk[path_index+1]
        xy_from, xy_to = XY[id_from], XY[id_to]
        dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
        distance = sqrt(dx * dx + dy * dy)
        pure_distance += distance
        if path_index % 10 == penalty_modulo and is_not_prime[id_from]:
            penalty += distance
    return pure_distance + 0.1 * penalty


@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def score_path(path):
    return score_chunk(0, path)


@numba.jit
def chunk_scores(chunk):
    scores = np.zeros(10)
    pure_distance = 0
    for i in numba.prange(chunk.shape[0] - 1):
        id_from, id_to = chunk[i], chunk[i+1]
        xy_from, xy_to = XY[id_from], XY[id_to]
        dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
        distance = sqrt(dx * dx + dy * dy)
        pure_distance += distance
        if is_not_prime[id_from]:
            scores[9-i%10] += distance
    scores *= 0.1
    scores += pure_distance
    return scores

def score_compound_chunk(offset, head, chunks, tail, scores, indexes_permutation=None):
    if indexes_permutation is None:
        indexes_permutation = range(len(chunks))
    score = 0.0
    last_city_id = head
    for index in indexes_permutation:
        chunk, chunk_scores = chunks[index], scores[index]
        score += cities_distance(offset % 10, last_city_id, chunk[0])
        score += chunk_scores[(offset + 1) % 10]
        last_city_id = chunk[-1]
        offset += len(chunk)
    return score + cities_distance(offset % 10, last_city_id, tail)

In [29]:
kdt = KDTree(XY)

In [30]:
triplets = set()
for city_id in tqdm(cities.index):
    dists, neibs = kdt.query([XY[city_id]], 7)
    for triplet in combinations(neibs[0], 3):
        if all(triplet):
            triplets.add(tuple(sorted(triplet)))
    neibs = kdt.query_radius([XY[city_id]], 10, count_only=False, return_distance=False)
    for triplet in combinations(neibs[0], 3):
        if all(triplet):
            triplets.add(tuple(sorted(triplet)))

print(f'{len(triplets)} cities triplets are selected.')

# sort triplets by distance
@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def sum_distance(ids):
    res = 0
    for i in numba.prange(len(ids)):
        for j in numba.prange(i + 1, len(ids)):
            res += cities_distance(0, ids[i], ids[j])
    return res

triplets = np.array(list(triplets))
distances = np.array(list(map(sum_distance, tqdm(triplets))))
order = distances.argsort()
triplets = triplets[order]


9355215 cities triplets are selected.


In [31]:
path = np.array(pd.read_csv('submission_close1.csv').Path)

In [32]:
def not_trivial_permutations(iterable):
    perms = permutations(iterable)
    next(perms)
    yield from perms


@lru_cache(maxsize=None)
def not_trivial_indexes_permutations(length):
    return np.array([list(p) for p in not_trivial_permutations(range(length))])

path_index = np.argsort(path[:-1])
print(f'Total score is {score_path(path):.2f}.')
for _ in range(3):
    for ids in tqdm(triplets):
        i, j, k = sorted(path_index[ids])
        head, tail = path[i-1], path[k+1]
        chunks = [path[i:i+1], path[i+1:j], path[j:j+1], path[j+1:k], path[k:k+1]]
        chunks = [chunk for chunk in chunks if len(chunk)]
        scores = [chunk_scores(chunk) for chunk in chunks]
        default_score = score_compound_chunk(i-1, head, chunks, tail, scores)
        best_score = default_score
        for indexes_permutation in not_trivial_indexes_permutations(len(chunks)):
            score = score_compound_chunk(i-1, head, chunks, tail, scores, indexes_permutation)
            if score < best_score:
                permutation = [chunks[i] for i in indexes_permutation]
                best_chunk = np.concatenate([[head], np.concatenate(permutation), [tail]])
                best_score = score
        if best_score < default_score:
            path[i-1:k+2] = best_chunk
            path_index = np.argsort(path[:-1])
            print(f'New total score is {score_path(path):.2f}. Permutating path at indexes {i}, {j}, {k}.')
    triplets = triplets[:10**6]

Total score is 1516230.52.


New total score is 1516230.09. Permutating path at indexes 160980, 160983, 160997.
New total score is 1516229.61. Permutating path at indexes 175466, 175468, 175480.



In [33]:
pd.DataFrame({'Path': path}).to_csv('submission_close1.csv', index=False)

In [ ]:
# https://www.kaggle.com/kostyaatarik/better-input-for-aka-2-opt
import numpy as np
import pandas as pd
import numba
from sympy import isprime, primerange
from math import sqrt
from sklearn.neighbors import KDTree
from tqdm import tqdm
from itertools import combinations

In [ ]:
cities = pd.read_csv('input/cities.csv', index_col=['CityId'])
XY = np.stack((cities.X.astype(np.float32), cities.Y.astype(np.float32)), axis=1)
is_not_prime = np.array([0 if isprime(i) else 1 for i in cities.index], dtype=np.int32)

In [ ]:
@numba.jit('f8(i8, i8, i8)', nopython=True, parallel=False)
def cities_distance(offset, id_from, id_to):
    xy_from, xy_to = XY[id_from], XY[id_to]
    dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
    distance = sqrt(dx * dx + dy * dy)
    if offset % 10 == 9 and is_not_prime[id_from]:
        return 1.1 * distance
    return distance


@numba.jit('f8(i4, i8[:])', nopython=True, parallel=False)
def score_chunk(offset, chunk):
    pure_distance, penalty = 0.0, 0.0
    penalty_modulo = 9 - offset % 10
    for path_index in numba.prange(chunk.shape[0] - 1):
        id_from, id_to = chunk[path_index], chunk[path_index+1]
        xy_from, xy_to = XY[id_from], XY[id_to]
        dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
        distance = sqrt(dx * dx + dy * dy)
        pure_distance += distance
        if path_index % 10 == penalty_modulo and is_not_prime[id_from]:
            penalty += distance
    return pure_distance + 0.1 * penalty


@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def score_path(path):
    return score_chunk(0, path)

In [ ]:
kdt = KDTree(XY)

In [ ]:
pairs = set()
for city_id in tqdm(cities.index):
    dists, neibs = kdt.query([XY[city_id]], 31)
    for neib_id in neibs[0][1:]:
        if city_id and neib_id:  # skip pairs that include starting city 
            pairs.add(tuple(sorted((city_id, neib_id))))
    neibs = kdt.query_radius([XY[city_id]], 31, count_only=False, return_distance=False)
    for neib_id in neibs[0]:
        if city_id and neib_id and city_id != neib_id:
            pairs.add(tuple(sorted((city_id, neib_id))))

print(f'{len(pairs)} cities pairs are selected.')
# sort pairs by distance
pairs = np.array(list(pairs))
distances = np.sum((XY[pairs.T[0]] - XY[pairs.T[1]])**2, axis=1)
order = distances.argsort()
pairs = pairs[order]

In [26]:
path = np.array(pd.read_csv('submission_close.csv').Path)

In [28]:
pd.DataFrame({'Path': path}).to_csv('submission_close.csv', index=False)